### Load Datasets

In [15]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from geodatasets import get_path
from function_06 import load_data_with_delimiters
from cleaning_func import quick_data_report
from function_06 import show_heads
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap, MarkerCluster
import re
import pyproj

### READ DATA

In [16]:
file_info = [
    ("../data/raw/accidents_21_23.csv", ',')

]

dfs = load_data_with_delimiters(file_info)

df_nl = dfs[0]
display(df_nl.head())


/Users/volkan.coskun/esra/data/Week_9/Prediction_Bike_Accident_New/notebook/function_06.py:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  return [pd.read_csv(path, delimiter=delim) for path, delim in file_info]


,FID,verkeersongeval_nummer,jaar_ongeval,verkeersongeval_afloop,aantal_partijen,partij_1_objecttype,partij_1_objecttype_overig,partij_2_objecttype,partij_2_objecttype_overig,aard_ongeval,...,woonplaats,actueel,wegbeheerder,gemeente,provincie,dienstnaam,districtnaam,indicatie_alcohol,gdb_geomattr_data,shape
0,ongevallen_2021_2023.1,20210000440,2021,Dodelijk,1,Bromfiets,NaN,NaN,NaN,Eenzijdig,...,Goedereede,Nee,Waterschap,Goeree-Overflakkee,Zuid-Holland,NaN,NaN,NaN,NaN,POINT (57047.666999999434 426867.4149999991)
1,ongevallen_2021_2023.2,20210075590,2021,Letsel,2,Personenauto,NaN,Personenauto,NaN,Flank,...,Assen,Nee,Gemeente,Assen,Drenthe,NaN,NaN,NaN,NaN,POINT (231850.2335000001 556678.6770000011)
2,ongevallen_2021_2023.3,20210077893,2021,Letsel,2,e-bike,NaN,Fiets,NaN,Onbekend,...,Zoetermeer,Ja,Gemeente,Zoetermeer,Zuid-Holland,NaN,NaN,NaN,NaN,POINT (94032.01949999854 452304.2441000007)
3,ongevallen_2021_2023.4,20210078473,2021,Uitsluitend materiele schade,1,Personenauto,NaN,NaN,NaN,Kop/staart,...,Leeuwarden,Nee,Rijk,Leeuwarden,Friesland,NaN,NaN,NaN,NaN,POINT (185669.7148000002 576049.5830000006)
4,ongevallen_2021_2023.5,20210080244,2021,Uitsluitend materiele schade,1,Bromfiets,NaN,NaN,NaN,Eenzijdig,...,Ritthem,Nee,Provincie,Vlissingen,Zeeland,NaN,NaN,NaN,NaN,POINT (33449.004000000656 388884.41600000113)


In [17]:
# Read KNMI weather file separately
weather_path = "../data/raw/etmgeg_2402.csv"

# Skip metadata (usually first 60 rows)
weather_df = pd.read_csv(weather_path, skiprows=51,low_memory=False)

# Preview
quick_data_report(weather_df)



DataFrame shape: (27170, 41)

Data types:
 # STN        int64
YYYYMMDD     int64
DDVEC        int64
FHVEC        int64
   FG        int64
  FHX        int64
 FHXH        int64
  FHN        int64
 FHNH        int64
  FXX       object
 FXXH       object
   TG        int64
   TN        int64
  TNH       object
   TX        int64
  TXH       object
 T10N       object
T10NH       object
   SQ       object
   SP       object
    Q       object
   DR       object
   RH       object
  RHX       object
 RHXH       object
   PG        int64
   PX       object
  PXH       object
   PN       object
  PNH       object
  VVN       object
 VVNH       object
  VVX       object
 VVXH       object
   NG       object
   UG       object
   UX       object
  UXH       object
   UN       object
  UNH       object
 EV24       object
dtype: object

Missing values:
 # STN       0
YYYYMMDD    0
DDVEC       0
FHVEC       0
   FG       0
  FHX       0
 FHXH       0
  FHN       0
 FHNH       0
  FXX       0
 FXXH

### CLEANING

In [18]:
#assign columns new name
weather_df.columns = [
    "station", "date", "wind_dir_deg", "wind_vec_0.1ms", "wind_avg_0.1ms", "wind_max_hourly_0.1ms", "wind_max_hourly_hour",
    "wind_min_hourly_0.1ms", "wind_min_hourly_hour", "wind_gust_max_0.1ms", "wind_gust_max_hour",
    "temp_avg_0.1C", "temp_min_0.1C", "temp_min_hour", "temp_max_0.1C", "temp_max_hour",
    "temp_10cm_min_0.1C", "temp_10cm_hour", "sunshine_duration_0.1h", "sunshine_percent", "radiation_jcm2",
    "rain_duration_0.1h", "precip_0.1mm", "precip_max_hour_0.1mm", "precip_max_hour",
    "pressure_avg_0.1hPa", "pressure_max_0.1hPa", "pressure_max_hour", "pressure_min_0.1hPa", "pressure_min_hour",
    "visibility_min_code", "visibility_min_hour", "visibility_max_code", "visibility_max_hour",
    "cloud_cover_octants", "humidity_avg_pct", "humidity_max_pct", "humidity_max_hour",
    "humidity_min_pct", "humidity_min_hour", "evapotranspiration_0.1mm"
]


In [19]:
weather_df.head()

,station,date,wind_dir_deg,wind_vec_0.1ms,wind_avg_0.1ms,wind_max_hourly_0.1ms,wind_max_hourly_hour,wind_min_hourly_0.1ms,wind_min_hourly_hour,wind_gust_max_0.1ms,...,visibility_min_hour,visibility_max_code,visibility_max_hour,cloud_cover_octants,humidity_avg_pct,humidity_max_pct,humidity_max_hour,humidity_min_pct,humidity_min_hour,evapotranspiration_0.1mm
0,240,19510101,188,77,87,195,18,41,24,,...,,,,7,90,98,6,73,20,
1,240,19510102,153,41,41,82,4,10,21,,...,,,,8,93,98,9,88,1,
2,240,19510103,203,15,21,51,24,0,3,,...,,,,6,94,100,21,83,12,
3,240,19510104,193,77,77,103,15,51,1,,...,,,,7,94,97,12,89,8,
4,240,19510105,207,82,87,144,21,36,24,,...,,,,8,95,100,5,89,14,


In [20]:
# Convert visibility columns to numeric
for col in ["visibility_min_code", "visibility_max_code"]:
    weather_df[col] = pd.to_numeric(weather_df[col], errors="coerce")

# Convert visibility codes to approximate meters using KNMI legend
# VVN/VVX code meaning:
#   0: <100 m, 1: 100–200 m, ..., 49: 4900–5000 m, 50: 5–6 km, ..., 89: >70 km
# We'll approximate midpoints for usable numeric values

def visibility_code_to_meters(code):
    if pd.isna(code):
        return None
    code = int(code)
    if code <= 49:
        return 100 + code * 100  # 0 = ~100m, 1 = 200m, ..., 49 = 5000m
    elif 50 <= code <= 55:
        return 5500 + (code - 50) * 1000  # 50 = 5.5km, 51 = 6.5km ...
    elif 56 <= code <= 79:
        return 6000 + (code - 56) * 1000  # up to 29-30km
    elif 80 <= code <= 89:
        return 32000 + (code - 80) * 5000  # 80 = 35km, 89 = 80km+
    else:
        return None

# Apply mapping
weather_df["visibility_min_m"] = weather_df["visibility_min_code"].apply(visibility_code_to_meters)
weather_df["visibility_max_m"] = weather_df["visibility_max_code"].apply(visibility_code_to_meters)

# Add to clean weather DataFrame
weather_df["visibility_min_m"] = weather_df["visibility_min_m"]
weather_df["visibility_max_m"] = weather_df["visibility_max_m"]



In [21]:
weather_df["date"] = pd.to_datetime(weather_df["date"], format="%Y%m%d")

for col in ["temp_avg_0.1C", "precip_0.1mm", "wind_avg_0.1ms", "sunshine_duration_0.1h", "humidity_avg_pct"]:
    weather_df[col] = pd.to_numeric(weather_df[col], errors="coerce")

# Create clean, scaled columns
weather_df["temp_C"] = weather_df["temp_avg_0.1C"] / 10
weather_df["precip_mm"] = weather_df["precip_0.1mm"] / 10
weather_df["wind_avg_ms"] = weather_df["wind_avg_0.1ms"] / 10
weather_df["sun_hours"] = weather_df["sunshine_duration_0.1h"] / 10
weather_df["humidity_pct"] = weather_df["humidity_avg_pct"]

# Final cleaned dataset
df_wea = weather_df[["date", "temp_C", "precip_mm", "wind_avg_ms", "sun_hours", "humidity_pct","visibility_min_m","visibility_max_m"]]


In [22]:
df_wea =df_wea[df_wea["date"] >= "2017-01-01"]


In [23]:
df_wea.head()

,date,temp_C,precip_mm,wind_avg_ms,sun_hours,humidity_pct,visibility_min_m,visibility_max_m
24107,2017-01-01,1.6,0.6,5.3,0.0,97.0,200.0,3800.0
24108,2017-01-02,3.2,2.0,3.3,4.6,83.0,2800.0,32000.0
24109,2017-01-03,5.7,0.4,7.2,0.7,85.0,11000.0,25000.0
24110,2017-01-04,5.9,0.6,8.0,2.8,76.0,3900.0,22000.0
24111,2017-01-05,0.3,-0.1,3.0,6.2,79.0,15000.0,47000.0


In [24]:
 df_nl.head()

,FID,verkeersongeval_nummer,jaar_ongeval,verkeersongeval_afloop,aantal_partijen,partij_1_objecttype,partij_1_objecttype_overig,partij_2_objecttype,partij_2_objecttype_overig,aard_ongeval,...,woonplaats,actueel,wegbeheerder,gemeente,provincie,dienstnaam,districtnaam,indicatie_alcohol,gdb_geomattr_data,shape
0,ongevallen_2021_2023.1,20210000440,2021,Dodelijk,1,Bromfiets,NaN,NaN,NaN,Eenzijdig,...,Goedereede,Nee,Waterschap,Goeree-Overflakkee,Zuid-Holland,NaN,NaN,NaN,NaN,POINT (57047.666999999434 426867.4149999991)
1,ongevallen_2021_2023.2,20210075590,2021,Letsel,2,Personenauto,NaN,Personenauto,NaN,Flank,...,Assen,Nee,Gemeente,Assen,Drenthe,NaN,NaN,NaN,NaN,POINT (231850.2335000001 556678.6770000011)
2,ongevallen_2021_2023.3,20210077893,2021,Letsel,2,e-bike,NaN,Fiets,NaN,Onbekend,...,Zoetermeer,Ja,Gemeente,Zoetermeer,Zuid-Holland,NaN,NaN,NaN,NaN,POINT (94032.01949999854 452304.2441000007)
3,ongevallen_2021_2023.4,20210078473,2021,Uitsluitend materiele schade,1,Personenauto,NaN,NaN,NaN,Kop/staart,...,Leeuwarden,Nee,Rijk,Leeuwarden,Friesland,NaN,NaN,NaN,NaN,POINT (185669.7148000002 576049.5830000006)
4,ongevallen_2021_2023.5,20210080244,2021,Uitsluitend materiele schade,1,Bromfiets,NaN,NaN,NaN,Eenzijdig,...,Ritthem,Nee,Provincie,Vlissingen,Zeeland,NaN,NaN,NaN,NaN,POINT (33449.004000000656 388884.41600000113)


In [25]:
#Drop unnecessary  columns

# 2) Pick only the columns you care about
keep_cols = [
    'verkeersongeval_nummer',
    'jaar_ongeval',
    'verkeersongeval_afloop',
    'aantal_partijen',
    'partij_1_objecttype',
    'partij_2_objecttype',
    'aard_ongeval',
    'wegsituatie',
    'bebouwde_kom',
    'maximum_snelheid',
    'wegverlichting',
    'wegverharding',
    'wegdek',
    'lichtgesteldheid',
    'weersgesteldheid',
    'straatnaam',
    'gemeente',
    'shape'
]
df_clean = df_nl[keep_cols].copy()

# 3) Rename to English
df_clean = df_clean.rename(columns={
    'verkeersongeval_nummer':   'accident_id',
    'jaar_ongeval':             'accident_year',
    'verkeersongeval_afloop':   'outcome',
    'aantal_partijen':          'num_parties',
    'partij_1_objecttype':      'party1_type',
    'partij_2_objecttype':      'party2_type',
    'aard_ongeval':             'accident_nature',
    'wegsituatie':              'road_situation',
    'bebouwde_kom':             'built_up_area',
    'maximum_snelheid':         'speed_limit_kmh',
    'wegverlichting':           'street_lighting',
    'wegverharding':            'road_surface',
    'wegdek':                   'road_condition',
    'lichtgesteldheid':         'light_condition',
    'weersgesteldheid':         'weather_condition',
    'straatnaam':               'street_name',
    'woonplaats':               'city',
    'gemeente':                 'municipality',
    'provincie':                'province',
    'shape':                    'shape'
})

# 4) (Optional) reset index
df_clean = df_clean.reset_index(drop=True)

# Take a quick look
print(df_clean.head())
print("\nCleaned shape:", df_clean.shape)


   accident_id  accident_year                       outcome  num_parties  \
0  20210000440           2021                      Dodelijk            1   
1  20210075590           2021                        Letsel            2   
2  20210077893           2021                        Letsel            2   
3  20210078473           2021  Uitsluitend materiele schade            1   
4  20210080244           2021  Uitsluitend materiele schade            1   

    party1_type   party2_type accident_nature  \
0     Bromfiets           NaN       Eenzijdig   
1  Personenauto  Personenauto           Flank   
2        e-bike         Fiets        Onbekend   
3  Personenauto           NaN      Kop/staart   
4     Bromfiets           NaN       Eenzijdig   

                    road_situation built_up_area  speed_limit_kmh  \
0                       Rechte weg        Buiten             60.0   
1              Kruispunt, 3 takken        Binnen             70.0   
2              Kruispunt, 3 takken       

In [26]:
# 1) Normalize text columns
for col in ['party1_type','party2_type','municipality']:
    df_clean[col] = df_clean[col].str.strip().str.lower()

# 2) Build the bike/e-bike-involved mask
bike_mask = (
    df_clean['party1_type'].str.contains(r'fiets|e-?bike', regex=True, na=False) |
    df_clean['party2_type'].str.contains(r'fiets|e-?bike', regex=True, na=False)
)

# 3) Amsterdam-only mask
ams_mask = df_clean['municipality'] == 'amsterdam'

# 4) Apply both
df_bike_ams = df_clean[bike_mask & ams_mask].copy()

print("Filtered shape:", df_bike_ams.shape)
print(df_bike_ams[['accident_id','party1_type','party2_type','municipality']].head())



Filtered shape: (5858, 18)
     accident_id   party1_type       party2_type municipality
58   20210077485  personenauto         bromfiets    amsterdam
91   20210081692  personenauto            e-bike    amsterdam
106  20210081708     snorfiets             fiets    amsterdam
154  20210080056        e-bike  landbouwvoertuig    amsterdam
155  20210080070  personenauto             fiets    amsterdam


In [27]:
df_bike_ams.head()

,accident_id,accident_year,outcome,num_parties,party1_type,party2_type,accident_nature,road_situation,built_up_area,speed_limit_kmh,street_lighting,road_surface,road_condition,light_condition,weather_condition,street_name,municipality,shape
58,20210077485,2021,Uitsluitend materiele schade,2,personenauto,bromfiets,Flank,"Kruispunt, 4 takken",Binnen,50.0,Brandend,NaN,Nat,Duisternis,Droog,Middenweg,amsterdam,POINT (124415.5298999995 484676.3922000006)
91,20210081692,2021,Uitsluitend materiele schade,2,personenauto,e-bike,Frontaal,"Kruispunt, 3 takken",Binnen,50.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Overschiestraat,amsterdam,POINT (117678.65309999883 484278.08260000125)
106,20210081708,2021,Uitsluitend materiele schade,2,snorfiets,fiets,Frontaal,Rechte weg,Binnen,50.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Rozenburglaan,amsterdam,POINT (124437.1950000003 483597.2190000005)
154,20210080056,2021,Uitsluitend materiele schade,2,e-bike,landbouwvoertuig,Flank,"Kruispunt, 4 takken",Binnen,50.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Th. K. van Lohuizenlaan,amsterdam,POINT (124961 486566)
155,20210080070,2021,Uitsluitend materiele schade,2,personenauto,fiets,Flank,"Kruispunt, 4 takken",Binnen,30.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Spuistraat,amsterdam,POINT (121107.48209999874 487274.5234000012)


In [28]:
quick_data_report(df_bike_ams)


DataFrame shape: (5858, 18)

Data types:
 accident_id            int64
accident_year          int64
outcome               object
num_parties            int64
party1_type           object
party2_type           object
accident_nature       object
road_situation        object
built_up_area         object
speed_limit_kmh      float64
street_lighting       object
road_surface          object
road_condition        object
light_condition       object
weather_condition     object
street_name           object
municipality          object
shape                 object
dtype: object

Missing values:
 accident_id            0
accident_year          0
outcome                0
num_parties            0
party1_type           25
party2_type          690
accident_nature        0
road_situation       236
built_up_area        217
speed_limit_kmh      473
street_lighting      284
road_surface         416
road_condition       233
light_condition        0
weather_condition    580
street_name            0
mun

In [29]:
df_bike_ams['party1_type']= df_bike_ams['party1_type'].fillna('unknown')
df_bike_ams['party2_type']= df_bike_ams['party2_type'].fillna('unknown')

In [30]:
cols = [
    'street_lighting',
    'light_condition',
    'weather_condition',
    'built_up_area',
    'speed_limit_kmh',
    'road_surface',
    'road_situation',
    'road_condition'     
]

# Fill NaNs with "Unknown"
df_bike_ams[cols] = df_bike_ams[cols].fillna('Unknown')


In [31]:
df_bike_ams.head()

,accident_id,accident_year,outcome,num_parties,party1_type,party2_type,accident_nature,road_situation,built_up_area,speed_limit_kmh,street_lighting,road_surface,road_condition,light_condition,weather_condition,street_name,municipality,shape
58,20210077485,2021,Uitsluitend materiele schade,2,personenauto,bromfiets,Flank,"Kruispunt, 4 takken",Binnen,50.0,Brandend,Unknown,Nat,Duisternis,Droog,Middenweg,amsterdam,POINT (124415.5298999995 484676.3922000006)
91,20210081692,2021,Uitsluitend materiele schade,2,personenauto,e-bike,Frontaal,"Kruispunt, 3 takken",Binnen,50.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Overschiestraat,amsterdam,POINT (117678.65309999883 484278.08260000125)
106,20210081708,2021,Uitsluitend materiele schade,2,snorfiets,fiets,Frontaal,Rechte weg,Binnen,50.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Rozenburglaan,amsterdam,POINT (124437.1950000003 483597.2190000005)
154,20210080056,2021,Uitsluitend materiele schade,2,e-bike,landbouwvoertuig,Flank,"Kruispunt, 4 takken",Binnen,50.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Th. K. van Lohuizenlaan,amsterdam,POINT (124961 486566)
155,20210080070,2021,Uitsluitend materiele schade,2,personenauto,fiets,Flank,"Kruispunt, 4 takken",Binnen,30.0,Niet brandend,Overig asfalt,Droog,Daglicht,Droog,Spuistraat,amsterdam,POINT (121107.48209999874 487274.5234000012)


In [32]:
# 1) Parse the WKT from your existing 'shape' column
df_bike_ams['geometry'] = df_bike_ams['shape'].apply(wkt.loads)

# 2) Turn into a GeoDataFrame using the Dutch national grid CRS (RD New)
gdf_ams = gpd.GeoDataFrame(df_bike_ams, geometry='geometry', crs='EPSG:28992')

# 3) Reproject to WGS84 (lat/lon)
gdf_ams = gdf_ams.to_crs(epsg=4326)

# 4) Extract lon/lat
gdf_ams['longitude'] = gdf_ams.geometry.x
gdf_ams['latitude']  = gdf_ams.geometry.y

# Quick check
print(gdf_ams[['longitude','latitude']].head())

     longitude   latitude
58    4.938332  52.349144
91    4.839506  52.345148
106   4.938748  52.339446
154   4.946168  52.366157
155   4.889521  52.372300


In [82]:
df_bike_ams.to_csv("../data/cleaned/accid_21_23.csv")
gdf_ams.to_csv("../data/cleaned/gdf_ams_21_23.csv")

In [34]:
gdf_ams.columns


Index(['accident_id', 'accident_year', 'outcome', 'num_parties', 'party1_type',
       'party2_type', 'accident_nature', 'road_situation', 'built_up_area',
       'speed_limit_kmh', 'street_lighting', 'road_surface', 'road_condition',
       'light_condition', 'weather_condition', 'street_name', 'municipality',
       'shape', 'geometry', 'longitude', 'latitude'],
      dtype='object')